# Assignmnent 1 DRM

#### by Dag Stabell Storhaug, Kristian Wang Høiem, Paul Joakim Andreassen, og Espen Sønneland

## Abstract



## The Code

In [1]:
"""
Putt inn coden her
"""

'\nPutt inn coden her\n'

## Task 1

In [2]:
"""
Putt inn coden her
"""

'\nPutt inn coden her\n'

In [35]:
import numpy as np
from scipy.optimize import linprog
from numpy.linalg import solve

hours = 24
peak_hour_from = 17
peak_hour_to = 20
num_appl = 3 # Number of appliances

dishwasher     = np.zeros(num_appl*hours) #[1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0]
washingmachine = np.zeros(num_appl*hours)  #[0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0]
EV             = np.zeros(num_appl*hours) #[0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1]

dishwasher[:24] = 1
washingmachine[24:48] = 1
EV[48:] = 1

TotalEnergyConsumption_dishwasher = 1.44 # kWh
TotalEnergyConsumption_washingmacine = 1.94  # kWh
TotalEnergyConsumption_EV = 9.9 # kWh


maxPowerUsage_dish = 1.8 # kW brukt per time
maxPowerUsage_wash = 0.5 # kW brukt per time
maxPowerUsage_EV = 3.6 # kW brukt per time 
maxPowerusage_total = 10 # kW brukt per time

price = np.ones(hours)*0.5
price[peak_hour_from:peak_hour_to] = 1 # peak-hour 5pm-8pm

A_eq = np.array([                        # Hvilke dingser som brukes i likhetsbegrensningen
                dishwasher,
                washingmachine,
                EV
])

b_eq = np.array([TotalEnergyConsumption_dishwasher,      # Summen av likhetsbegresningen
                 TotalEnergyConsumption_washingmacine,
                 TotalEnergyConsumption_EV
])


A_mul = np.zeros((hours,num_app*hours))
for i in range(A_mul.shape[0]):
    A_mul[i,i::hours] = 1
A_one = np.eye(num_appl*hours)    
A_ub = np.concatenate((A_one,A_mul),axis=0)  # Hvilket timesobjekt begrensningen hører til 


b_one_ub = np.array( [maxPowerUsage_dish]*hours    # Begrensning
                    +[maxPowerUsage_wash]*hours
                    +[maxPowerUsage_EV]*hours
)
b_max_ub = np.array([maxPowerusage_total]*hours)

b_ub = np.concatenate((b_one_ub,b_max_ub)) 

c = np.array([])  #np.array([p1,p2,p3,p4,p1,p2,p3,p4]) Coefficients of the linear objective function to be minimized
for _ in range(num_appl):
    c = np.append(price,c)

res = linprog(c, A_eq=A_eq, b_eq=b_eq, A_ub=A_ub, b_ub=b_ub, bounds=(0, None))
print('Optimal value:', res.fun, '\nX:\n', res.x.reshape(3,24))

Optimal value: 6.640000000000001 
X:
 [[1.44 0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.
  0.   0.   0.   0.   0.   0.   0.   0.   0.   0.  ]
 [0.5  0.5  0.5  0.44 0.   0.   0.   0.   0.   0.   0.   0.   0.   0.
  0.   0.   0.   0.   0.   0.   0.   0.   0.   0.  ]
 [3.6  3.6  2.7  0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.
  0.   0.   0.   0.   0.   0.   0.   0.   0.   0.  ]]


In [32]:
maxPowerusage_total = 10
b_max_ub = np.array([maxPowerusage_total]*hours)
b_one_ub = np.array( [maxPowerUsage_dish]*hours    # Begrensning
                +[maxPowerUsage_wash]*hours
                +[maxPowerUsage_EV]*hours
)
b_ub = np.concatenate((b_one_ub,b_max_ub)) 
b_ub

array([ 1.8,  1.8,  1.8,  1.8,  1.8,  1.8,  1.8,  1.8,  1.8,  1.8,  1.8,
        1.8,  1.8,  1.8,  1.8,  1.8,  1.8,  1.8,  1.8,  1.8,  1.8,  1.8,
        1.8,  1.8,  0.5,  0.5,  0.5,  0.5,  0.5,  0.5,  0.5,  0.5,  0.5,
        0.5,  0.5,  0.5,  0.5,  0.5,  0.5,  0.5,  0.5,  0.5,  0.5,  0.5,
        0.5,  0.5,  0.5,  0.5,  3.6,  3.6,  3.6,  3.6,  3.6,  3.6,  3.6,
        3.6,  3.6,  3.6,  3.6,  3.6,  3.6,  3.6,  3.6,  3.6,  3.6,  3.6,
        3.6,  3.6,  3.6,  3.6,  3.6,  3.6, 10. , 10. , 10. , 10. , 10. ,
       10. , 10. , 10. , 10. , 10. , 10. , 10. , 10. , 10. , 10. , 10. ,
       10. , 10. , 10. , 10. , 10. , 10. , 10. , 10. ])

### Discution

## Task 2

In [3]:
"""
Putt inn coden her
"""

'\nPutt inn coden her\n'

### Discution

## Task 3

In [4]:
"""
Putt inn coden her
"""

'\nPutt inn coden her\n'

### Discution

## Conclution